In [1]:
from Dataset import CustomDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from model import RiceDetector
from torchsummary import summary
import torch.optim
import torch.nn as nn
from tqdm import tqdm

In [2]:
dataset = CustomDataset("Data/labels/label_csv.csv", "Data/images/")
len(dataset)

75000

In [3]:
train_len = int(len(dataset) * 0.8)
val_len = int(len(dataset) * 0.1)
test_len = int(len(dataset) * 0.1)
print(train_len)
print(val_len)
print(test_len)
train_data, val_data, test_data = random_split(dataset, lengths=[train_len, val_len, test_len])

60000
7500
7500


In [4]:
train_data.__getitem__(0)[0].shape

torch.Size([3, 250, 250])

In [5]:
train_loader, val_loader, test_loader = DataLoader(train_data, 2), DataLoader(val_data, 2), DataLoader(test_data, 1)

In [6]:
model = RiceDetector()
model.to("cuda")

RiceDetector(
  (conv_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool_1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (pool_2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv_3): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
  (pool_3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv_4): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool_4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc_1): Linear(in_features=21632, out_features=1024, bias=True)
  (fc_2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc_3): Linear(in_features=1024, out_features=5, bias=True)
  (relu): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (batch_norm_1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [7]:
def train_loop(loss_fn, optimizer):
    for x_batch, y_batch in tqdm(train_loader, desc="Train loop: "):
        optimizer.zero_grad()
        x_batch = x_batch.to("cuda")
        y_batch = y_batch.to("cuda")
        y_batch = nn.functional.one_hot(y_batch, 5).float().to("cuda")
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
    
    return loss
        

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
model.train()
for epoch in tqdm(range(0, 5), desc="epoch: "):
    epoch_loss = train_loop(loss_fn, optimizer)
    print("epoch {} , Loss: {}".format(epoch, epoch_loss))
    

epoch:  20%|██        | 1/5 [09:43<38:55, 583.83s/it]

epoch 0 , Loss: 2.1817030906677246


epoch:  40%|████      | 2/5 [19:28<29:12, 584.25s/it]

epoch 1 , Loss: 2.1341142654418945


epoch:  40%|████      | 2/5 [19:32<29:18, 586.18s/it]


KeyboardInterrupt: 